In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
import sys
os.chdir('/content/drive/My Drive/ProyectoIA2/generative-spectral-datasets')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
import numpy as np
from models.main import make_autoencoder, make_discriminator

input_shape = (32,32,3)

In [3]:
(train_images, train_labels), (_, _) = tf.keras.datasets.cifar10.load_data()

train_images = train_images.reshape(train_images.shape[0], 32, 32, 3).astype('float32')
train_images = train_images / 255

BUFFER_SIZE = 60000
BATCH_SIZE = 256

train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [4]:
mse_loss = tf.keras.losses.MeanSquaredError()


def discriminator_loss(real_output, fake_output):
    loss = tf.math.log(real_output) + tf.math.log(1 - fake_output)
    return - tf.math.reduce_mean(loss)
    
def autoencoder_loss(real_x, estimated_x, real_output, fake_output, tau=0.01):
    autoencoder_loss = mse_loss(real_x, estimated_x)
    gan_loss =  tau*discriminator_loss(real_output, fake_output)
    total_loss = autoencoder_loss - gan_loss
    return total_loss

autoencoder_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

FACTOR = [ 1 , 1 , 1/16 , 1/32 , 1/64, 1/256]
features= 256

autoencoder = make_autoencoder(input_shape, features=features, factors=FACTOR)

# discriminator = make_discriminator(input_shape, features=4)
discriminator = tf.keras.applications.mobilenet.MobileNet(input_shape=input_shape, classes=1, weights=None, classifier_activation='sigmoid')



In [5]:
autoencoder.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
encoder (Encoder)            (None, 32, 32, 1)         637781    
_________________________________________________________________
generator (Generator)        (None, 32, 32, 3)         637779    
Total params: 1,275,560
Trainable params: 1,273,398
Non-trainable params: 2,162
_________________________________________________________________


In [6]:
discriminator.summary()

Model: "mobilenet_1.00_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 16, 16, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 16, 16, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 16, 16, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 16, 16, 32)        128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 16, 16, 32)  

In [7]:
@tf.function
def train_step(images):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

      generated_images = autoencoder(images, training=True)
      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = autoencoder_loss(images, generated_images, real_output, fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

      gradients_of_generator = gen_tape.gradient(gen_loss, autoencoder.trainable_variables)
      gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

      autoencoder_optimizer.apply_gradients(zip(gradients_of_generator, autoencoder.trainable_variables))
      discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

      return gen_loss , disc_loss


def train(dataset, epochs):
  print("START TRAINING")
  for epoch in range(epochs):

    ae_total_loss = []
    disc_total_loss = []

    for image_batch in dataset:
      gen_loss , disc_loss = train_step(image_batch)

      ae_total_loss.append(float(gen_loss))
      disc_total_loss.append(float(disc_loss))
      print("Autoencoder loss: %.4f | Discriminator loss: %.4f" % (float(gen_loss), float(disc_loss))) 

    # ae_total_loss = np.mean(ae_total_loss)
    # disc_total_loss = np.mean(disc_total_loss)

    # print("Autoencoder loss: %.4f | Discriminator loss: %.4f" % (float(ae_total_loss), float(ae_total_loss))) 

  print("END TRAINING")


In [ ]:
EPOCHS = 10
train(train_dataset, EPOCHS)

START TRAINING


In [ ]:
x = next(iter(train_dataset))

In [ ]:
x_estimated = autoencoder.predict(x)

import matplotlib.pyplot as plt

num = np.random.randint(0, x.shape[0])

plt.imshow(x[num,:,:])
plt.show()

plt.imshow(x_estimated[num,:,:])
plt.show()